# Data Extraction

In [ ]:
# Normal imports
import warnings
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

#Specific imports
import rtree
import pygeos
import geopandas
from geopandas import GeoDataFrame
from shapely import wkt
from shapely.geometry import Point

#Internal imports
from src.data_extraction.data_extraction import extract_initial_data
from src.constants import (
    NEW_COLUMNS_NAMES,
    REMOVE_COLUMNS_BY_INPUT,
    REMOVE_COLUMNS_BY_CORRELATIONS,
    NUM_VARIABLES_TO_SEE_DISTRIBUTION,
    BINARY_VARIABLES,
    IDEALISTA_COLORS
)
from src.preprocessing.preprocessing_utils import (
    generate_pandas_profiling_report,
    remove_duplicated_anuncios_id,
    find_single_value_columns,
    treatment_missing_values,
    visualize_distribution,
    visualize_binary_distribution,
    correlation_values,
    feature_engineering,
    get_location_name_w_gdf,
)

# Settings
warnings.filterwarnings("ignore")
%matplotlib inline
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

## Extract all dataset available, provided by idealista

In [ ]:
df_assets, df_ine, df_osm, df_pois, df_polygons = extract_initial_data(
    root_dir="input_data"
)

In [ ]:
# Change columns names to friendly ones
df_assets = df_assets.drop(columns=["ADTYPOLOGY", "ADOPERATION"])
df_assets.columns = NEW_COLUMNS_NAMES

## Polygons and INE censal polygons

#### Polygons

In [ ]:
# Convert WKT strings to Shapely geometries and create a GeoDataFrame
df_polygons['geometry'] = df_polygons['WKT'].apply(wkt.loads)
gdf_polygons = geopandas.GeoDataFrame(df_polygons['geometry'], crs='epsg:4326')

# Add additional columns to the GeoDataFrame
gdf_polygons['barrio_id'] = df_polygons['LOCATIONID']
gdf_polygons['barrio'] = df_polygons['LOCATIONNAME']

# Create Point geometries using longitude and latitude coordinates from df_assets
geometry = [Point(xy) for xy in zip(df_assets.longitud, df_assets.latitud)]

# Create a GeoDataFrame gdf_ads with df_prices data and geometry column
gdf_assets = GeoDataFrame(df_assets, crs="EPSG:4326", geometry=geometry)

# Apply a logarithmic scale transformation to the 'precio' column in gdf_ads
gdf_assets['precio_logaritmico'] = np.log(gdf_assets['precio'])


#### INE Censal Polygons

In [ ]:
# Convert WKT strings to Shapely geometries and create a GeoDataFrame for census polygons
df_ine['geometry'] = df_ine['WKT'].apply(wkt.loads)
gdf_polygons_census = geopandas.GeoDataFrame(df_ine['geometry'], crs='epsg:4326')

# Add additional column 'CUSEC' to the GeoDataFrame representing census polygons
gdf_polygons_census['cusec'] = df_ine['CUSEC']

In [ ]:
# Add the census codes (CUSEC)
gdf_assets = geopandas.sjoin(gdf_assets, gdf_polygons_census, how="inner")

# Drop index_right 
gdf_assets = gdf_assets.drop(columns=['index_right'])

# Now add the idealista zones (LOCATIONID, LOCATIONNAME)
gdf_assets = geopandas.sjoin(gdf_assets, gdf_polygons, how="inner")

# Drop index_right 
gdf_assets = gdf_assets.drop(columns=['index_right'])

df_assets = gdf_assets.copy()

## ASSETS - Basis

### General

In [ ]:
# Generate pandas profiling: select all columns except those selected
#FIXME: uncomment before commit
# generate_pandas_profiling_report(df=df_assets[df_assets.columns.difference(['geometry','cusec', 'barrio_id'])])

In [ ]:
# See general statistics of df
description_df = df_assets.describe().transpose().style.format("{:.2f}")
description_df

In [ ]:
# Remove duplicated anuncios_id
df_assets = remove_duplicated_anuncios_id(df_assets=df_assets, criteria="last")

In [ ]:
# Remove columns by input
df_assets = df_assets.drop(columns=REMOVE_COLUMNS_BY_INPUT)
print('Removed columns:', REMOVE_COLUMNS_BY_INPUT)

In [ ]:
# Remove columns that only have one different value
remove_unique_value_columns = find_single_value_columns(df=df_assets)
df_assets = df_assets.drop(columns=remove_unique_value_columns)

### Missing Values

In [ ]:
# Missing values
remove_columns_by_missing_values, df_assets = treatment_missing_values(df=df_assets)

### Data Treatment and Feature Engineering

In [ ]:
# Mean price by barrio
# TODO: if needed, do the same to cusec
df_metrics_barrios= df_assets.groupby(['barrio']).agg({'precio':['median', 'mean', 'std'], 
                                   'precio_unitario_m2':['median', 'mean', 'std']}).reset_index()

df_metrics_barrios.columns = ['barrio', 
                         'precio_median_barrio', 'precio_mean_barrio', 'precio_std_barrio', 
                         'precio_unitario_m2_median_barrio', 'precio_unitario_m2_mean_barrio', 'precio_unitario_m2_std_barrio']

In [ ]:
# Add mean price by barrio to dataset
df_assets=df_assets.merge(df_metrics_barrios[['barrio', 'precio_mean_barrio', 'precio_unitario_m2_mean_barrio']], on=['barrio'], how="inner")

In [ ]:
# Remove this observation because it is a outlier: seen in graph in section "coordenadas"
df_assets = df_assets[df_assets['latitud'] >= 37]

In [ ]:
# Remove columns by high correlations
df_assets = df_assets.drop(columns=REMOVE_COLUMNS_BY_CORRELATIONS)

In [ ]:
# Feature Engineering: add new variables
(
    add_columns,
    remove_columns_by_creating_new_variables,
    df_assets,
) = feature_engineering(df=df_assets)

print("Columns added:", add_columns)
print('Columns removed by adding new:', remove_columns_by_creating_new_variables)

### TODO:
* Baños 0: Intentar imputar el numero de baños por el número de baño médio por metro cuadrado?
* Duplex: Hacer analysis de como es de distincto el precio entre casas duplex y no duplex
* Estudio: 
    1. Hacer analysis de como es de distincto el precio entre casas estudio y no estudio
    2. Mirar el numero de habitaciones de los estudios  
* Precio Parking:
    1. Validar el sentido de una casa sin parking tener precio del parking > 0 
    2. Validar porque todos los parking têm precio 1 y no 0
* Orientaciones: hay casas sin ninguna orientacion y no son solo interiores. 


N_Banos: Impute values when n_banos = 0

In [ ]:
# Filter rows where n_banos > 0
filtered_df = df_assets[df_assets['n_banos'] > 0]

# Calculate number of bathrooms per square meter
filtered_df['n_banos_m2'] = filtered_df['n_banos'] / filtered_df['area_construida']

# Calculate mean number of bathrooms per square meter
mean_bathrooms_per_sqm = filtered_df['n_banos_m2'].mean()

# Impute number of bathrooms for rows where n_banos == 0: mean number of bath by m^2 * m^2, rounded, and minimum 1
df_assets['n_banos_m2'] = (np.maximum(mean_bathrooms_per_sqm * df_assets['area_construida'],1)).round().astype(int)

# Validate if it is a good way to values, calculating mape comparing with the real n_banos
df_assets_aux = df_assets[df_assets['n_banos']>0]
absolute_percentage_errors = np.abs((df_assets_aux['n_banos'] - df_assets_aux['n_banos_m2']) / df_assets_aux['n_banos'])
mape = np.mean(absolute_percentage_errors) * 100
print("MAPE", mape)

# Delete filtered_df, df_assets_aux as they are only auxiliar 
del filtered_df, df_assets_aux

# Assign the imputed value to n_banos where n_banos == 0
df_assets.loc[df_assets['n_banos'] == 0, 'n_banos'] = df_assets.loc[df_assets['n_banos'] == 0, 'n_banos_m2']

# # Drop the n_banos_m2 column as it is no longer needed
df_assets = df_assets.drop(columns=['n_banos_m2'])

In [ ]:
# UTIL
df_assets.to_csv(r'output_data\df_util.csv')

Duplex

In [ ]:
# Calculate mean price for duplex and non-duplex houses
mean_price_duplex = df_assets[df_assets.duplex==1]['precio'].mean()
mean_price_no_duplex = df_assets[df_assets.duplex!=1]['precio'].mean()
print('Duplex prices are, in average,', round(mean_price_duplex/mean_price_no_duplex,2), 'times the price of a non duplex house')

Studio

In [ ]:
# Calculate mean price for duplex and non-duplex houses
mean_price_estudio = df_assets[df_assets.estudio==1]['precio'].mean()
mean_price_no_estudio = df_assets[df_assets.estudio!=1]['precio'].mean()
print('Studio prices are, in average,', round(mean_price_estudio/mean_price_no_estudio,2), 'times the price of a non studio house')

print('Studios distinct number of bathrooms:', 
      df_assets[df_assets.estudio==1][['n_banos']].value_counts())

Precio Parking

    1. si precio_parking>1 --> parking = 1 
    2. si parking==0 --> precio_parking = 0 
    
Validar porque tenemos precio_parking = 1 cuando parking = 0


In [ ]:
df_assets.loc[(df_assets['parking'] == 0) & (df_assets['precio_parking'] > 1), 'precio_parking'].value_counts()

Orientaciones

In [ ]:
# Calculate the sum of all orientations to see if we have information about orientation of all houses
df_assets['orientacion_total'] = df_assets.orientacion_s+df_assets.orientacion_e+df_assets.orientacion_n+df_assets.orientacion_o

# Create a cross-tabulation between 'orientacion_total' and 'interior'
cross_tab = pd.crosstab(df_assets['orientacion_total'], df_assets['interior'])
print('Compare total orientation with if is an interior house or not')
print(cross_tab)

### Correlations

In [ ]:
# Correlation values
correlation_matrix, correlated_variables = correlation_values(df=df_assets, threshold=0.8)

### Visualizations

In [ ]:
# visualize_distribution(
#     df=df_assets, numerical_columns=NUM_VARIABLES_TO_SEE_DISTRIBUTION
# )


#### Mean average price duplex vs non duplex

In [ ]:
# Create a bar plot
plt.figure(figsize=(8, 6))
bars = plt.bar(['Duplex', 'Non-Duplex'], [mean_price_duplex, mean_price_no_duplex], color=['blue', 'orange'])

# Add labels to each bar
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval, round(yval, 0).astype(int), va='bottom')

plt.xlabel('House Type')
plt.ylabel('Mean Price')
plt.title('Mean Price Comparison between Duplex and Non-Duplex Houses')
plt.show()

#### Price Density (Precio del Vuelo)

In [ ]:
# Price density
df_assets.precio.plot.kde()

In [ ]:
# Price density droping by 1M€
ax = df_assets.precio.plot.kde()
plt.xlim([0, 1000000])
plt.show()

In [ ]:
# Change scale of the price
df_assets['precio_logaritmico'] = np.log(df_assets['precio'])
ax = df_assets.precio_logaritmico.plot.kde()
plt.show()


Temas a tener en cuenta:

* La distribución de valores es multimodal
* La distribución de valores no es simétrica
* El rango de valores puede ser muy amplio

El precio depende de muchos factores, pero en la literatura existen dos grandes factores:

* Precio del suelo (el suelo donde está construido)
* Precio del vuelo (lo que está construido)

Para empezar, una forma de controlar el precio del suelo es incorporar información de la zona y una forma para controla el precio del suelo es normalizar por metros cuadrados (es nuestra variable __UNITPRICE__).

En la siguiente gráfica observamos el fenómeno de la multimodalidad, significa que podemos encontrarnos inmuebles con las mismas características constructivas con distintos precios €/m², ¿por qué?, principalmente por el otro factor: __el precio del suelo__.

#### Unit Price Density (Precio del suelo)

In [ ]:
ax = df_assets.precio_unitario_m2.plot.kde()
plt.xlim([0, 10000])
plt.show()

#### Mean price by Barrio

In [ ]:
# Sort DataFrame by highest mean price
df_metrics_barrios_sorted = df_metrics_barrios.sort_values(by='precio_mean', ascending=False)

# Set the custom color palette
sns.set_palette(IDEALISTA_COLORS)

# Plot mean price by barrio (sorted)
plt.figure(figsize=(20, 10))
plt.bar(df_metrics_barrios_sorted['barrio'], df_metrics_barrios_sorted['precio_mean'], color='skyblue')
plt.xlabel('Barrio')
plt.ylabel('Mean Price')
plt.title('Mean Price by Barrio (Ordered by Highest Price)')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()


#### Coordenadas

In [ ]:
# Price Total €
cm = plt.cm.get_cmap('magma')  # or your colormap of choice
ax = df_assets.plot.scatter(x='longitud', y='latitud', c='precio',figsize=(10, 10), cmap=cm)
ax.set_xlabel('Longitud', fontsize=15)
ax.set_ylabel('Latitud', fontsize=15)
ax.set_title('Madrid: Total price')
ax.figure.show()

# Price Total by km^2 €
cm = plt.cm.get_cmap('magma')  # or your colormap of choice
ax = df_assets.plot.scatter(x='longitud', y='latitud', c='precio_unitario_m2',figsize=(10, 10), cmap=cm)
ax.set_xlabel('Longitud', fontsize=15)
ax.set_ylabel('Latitud', fontsize=15)
ax.set_title('Madrid: Total price by m^2')
ax.figure.show()

#### Polygons

In [ ]:
# Plot the GeoDataFrame gdf_polygons as white polygons with black edges on the base plot
base = gdf_polygons.plot(color='white', edgecolor='black', figsize=(10, 10))

# Overlay gdf_ads on the same plot, plotting points colored by 'precio_unitario_km2' column
gdf_assets.plot(ax=base, marker='o', column='precio_unitario_m2', markersize=5, cmap='inferno')

In [ ]:
# Use logaritmic price
# Plot the GeoDataFrame gdf_polygons as white polygons with black edges on the base plot
base = gdf_polygons.plot(color='white', edgecolor='black', figsize=(10, 10))

# Overlay gdf_ads on the same plot, plotting points colored by 'precio_logaritmico' column
gdf_assets.plot(ax=base, marker='o', column='precio_logaritmico', markersize=5, cmap='inferno')


#### INE Censal Polygons

In [ ]:
# Plot the census polygons as white polygons with black edges on the base plot
base = gdf_polygons_census.plot(color='white', edgecolor='black', figsize=(10, 10))

# Overlay gdf_ads on the same plot, plotting points colored by 'precio_unitario_km2' column
gdf_assets.plot(ax=base, marker='o', column='precio_unitario_m2', markersize=5, cmap='inferno')


#### Points of Interest - Open Street Map - Basis

In [ ]:
# Show points of interest
unique_codes = df_osm['CODE'].unique()
cmap = plt.cm.get_cmap('magma', len(unique_codes))

# Create scatter plot
fig, ax = plt.subplots(figsize=(10, 10))
for i, code in enumerate(unique_codes):
    subset = df_osm[df_osm['CODE'] == code]
    ax.scatter(subset['LNG'], subset['LAT'], c=cmap(i), label=code)

# Set labels and title
ax.set_xlabel('Longitud', fontsize=15)
ax.set_ylabel('Latitud', fontsize=15)
ax.set_title('Points of Interest - Open Street Map')

# Add legend
ax.legend()
plt.show()
